In [ ]:
# pip install krpc

In [ ]:
import krpc
from krpc.services.spacecenter import SASMode
import time

In [ ]:
# Подключаемся к KSP
conn = krpc.connect(name='Launch into orbit')

!!!ЗАПУСКАТЬ ТОЛЬКО НА ЗАПУСКАТЕЛЬНОЙ ПЛОЩАДКЕ

In [ ]:
# Получаем объект судна
vessel = conn.space_center.active_vessel
auto_pilot = vessel.auto_pilot

In [ ]:
# Получаем потоки высоты и апоапсиса
altitude = conn.add_stream(getattr, vessel.flight(), 'mean_altitude')
apoapsis = conn.add_stream(getattr, vessel.orbit, 'apoapsis_altitude')

ПОДГОТОВКА К ПОЛЁТУ

In [ ]:
# Включаем SAS, RCS. Выставляем тягу на 50%
vessel.control.sas = True
vessel.control.rcs = True
vessel.control.throttle = 0.5

1 СТУПЕНЬ

In [ ]:
# Запускаем 1 ступень
vessel.control.activate_next_stage()

# Включаем радиальный режим SAS
vessel.control.sas_mode = SASMode.radial

# Уменьшаем/увеличиваем тягу в зависимости от перегрузки
g_force_stream = conn.add_stream(getattr, vessel.flight(), 'g_force')

stage_3_resources = vessel.resources_in_decouple_stage(stage=3, cumulative=False)

liquid_fuel = conn.add_stream(stage_3_resources.amount, 'LiquidFuel')
while liquid_fuel() > 0.1:
    if g_force_stream() > 2:
        vessel.control.throttle -= 0.1

    if g_force_stream() < 1.2:
        vessel.control.throttle += 0.1
    
    time.sleep(1)

    continue

2, 3 СТУПЕНИ: 2 Твердотопливных двигателя

In [ ]:
# Функция чтобы дождаться отработки ступени с твёрдым топливом
def wait_till_solid_fuel_end_stage(stage):
    stage_0_resources = vessel.resources_in_decouple_stage(stage=stage, cumulative=False)
    srb_fuel = conn.add_stream(stage_0_resources.amount, 'SolidFuel')

    while srb_fuel() > 0.1:
        continue

In [ ]:
# Устанавливаем значение тяги в 1 и режим SAS в stability_assist
vessel.control.throttle = 1
vessel.control.sas_mode = SASMode.stability_assist

In [ ]:
# Запускаем и дожидаемся окончания 2 ступени
vessel.control.activate_next_stage()
wait_till_solid_fuel_end_stage(2)

In [ ]:
# Поворачиваемся на 45 градусов
auto_pilot.engage()

auto_pilot.target_pitch_and_heading(45, 90)
auto_pilot.wait()

time.sleep(10)
auto_pilot.disengage()

vessel.control.sas = True
vessel.control.sas_mode = vessel.control.sas_mode.stability_assist

In [ ]:
# Запускаем и дожидаемся окончания 3 ступени
vessel.control.activate_next_stage()
wait_till_solid_fuel_end_stage(1)

In [ ]:
# Отсоединяем ступень
vessel.control.activate_next_stage()

4 СТУПЕНЬ

In [ ]:
# Ожидаем пока ракета подлетит к апоапсису
start_time = time.time()

while True:
    print(f'check {int(time.time() - start_time)}')
    if abs(apoapsis() - altitude()) < 3_000:
        break

In [ ]:
# Переводим ракету в горизонтальное положение

auto_pilot.engage()
auto_pilot.target_pitch_and_heading(0, 90)
time.sleep(10)
auto_pilot.disengage()

vessel.control.sas = True
vessel.control.sas_mode = vessel.control.sas_mode.stability_assist

In [ ]:
# Запускаем и дожидаемся окончания 4 ступени
vessel.control.activate_next_stage()
wait_till_solid_fuel_end_stage(-1)

In [ ]:
print('Запуск завершён')